In [0]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import normalize

df = pd.read_csv('./drive/My Drive/data/esc-50/meta/esc50.csv')
x_mfcc_mean = joblib.load('./drive/My Drive/data/esc-50/x_mfcc_mean.pkl')
x_spec_mean = joblib.load('./drive/My Drive/data/esc-50/x_spec_mean.pkl')
x_mfcc = joblib.load('./drive/My Drive/data/esc-50/x_mfcc.pkl')
x_spec = joblib.load('./drive/My Drive/data/esc-50/x_spec.pkl')
x_mfcc_feat = joblib.load('./drive/My Drive/data/esc-50/x_mfcc_feat.pkl')

X = np.concatenate((x_mfcc_mean, x_spec_mean, x_mfcc, x_spec), axis=1)
y = df['category']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.1)

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
rf_1 = RandomForestClassifier(max_depth = 72,
                            n_estimators = 1234,
                            min_samples_split = 2,
                            min_samples_leaf = 1,
                            n_jobs=-1)

rf_1.fit(X_train[:,:40], y_train)
p_1 = rf_1.predict_proba(X_train[:,:40])
p_1_test = rf_1.predict_proba(X_test[:,:40])
rf_1.score(X_test[:,:40], y_test)

0.535

In [5]:
rf_2 = RandomForestClassifier(max_depth = 200, n_jobs=-1, n_estimators=1000)

rf_2.fit(X_train[:,40:168], y_train)
p_2 = rf_2.predict_proba(X_train[:,40:168])
p_2_test = rf_2.predict_proba(X_test[:,40:168])
rf_2.score(X_test[:,40:168], y_test)

0.445

In [6]:
rf_3 = RandomForestClassifier(n_jobs=-1, n_estimators=1000)

rf_3.fit(X_train[:, 168:8808], y_train)
p_3 = rf_3.predict_proba(X_train[:, 168:8808])
p_3_test = rf_3.predict_proba(X_test[:, 168:8808])
rf_3.score(X_test[:, 168:8808], y_test)

0.495

In [7]:
rf_4 = RandomForestClassifier(n_jobs=-1, n_estimators=1000)

rf_4.fit(X_train[:, 8808:], y_train)
p_4 = rf_4.predict_proba(X_train[:, 8808:])
p_4_test = rf_4.predict_proba(X_test[:, 8808:])
rf_4.score(X_test[:, 8808:], y_test)

0.435

In [8]:
rf_5 = RandomForestClassifier(n_jobs=-1, n_estimators=1000)

rf_5.fit(X_train, y_train)
p_5 = rf_5.predict_proba(X_train)
p_5_test = rf_5.predict_proba(X_test)
rf_5.score(X_test, y_test)

0.485

In [0]:
p_train = np.concatenate((p_1, p_2, p_3, p_4, p_5), axis=1)
p_test = np.concatenate((p_1_test, p_2_test, p_3_test, p_4_test, p_5_test), axis=1)

In [10]:
rf = RandomForestClassifier(n_jobs=-1, n_estimators=1000)

rf.fit(p_train, y_train)
rf.score(p_test, y_test)

0.345

In [11]:
p_2_test.shape

(200, 50)

In [12]:
!pip install scikit-optimize
from skopt import gp_minimize
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score


def treinar_modelo(params):
    learning_rate = params[0]
    num_leaves = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    
    print(params, '\n')
    
    mdl = LGBMClassifier(learning_rate=learning_rate,
                         num_leaves=num_leaves,
                         min_child_samples=min_child_samples,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree,
                         random_state=0,
                         subsample_freq=1,
                         n_estimators=100)
    mdl.fit(X_train[:,40:168], y_train)
    
    y_pred = mdl.predict(X_test[:,40:168])
    
    return -accuracy_score(y_test,y_pred)

     |████████████████████████████████| 81kB 2.7MB/s 


In [0]:
space = [(1e-3, 1e-1, 'log-uniform'), #learning rate
         (2, 128), # num_leaves
         (1, 100), # min_child_samples
         (0.05, 1.0), # subsample
         (0.1, 1.0)] # colsample bytree


resultados_gp = gp_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=60, n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.
[0.09871192514273254, 120, 14, 0.9990884895579377, 0.3124800792567785] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 59.7461
Function value obtained: -0.4350
Current minimum: -0.4350
Iteration No: 2 started. Evaluating function at random point.
[0.006210998932353835, 51, 67, 0.9387621172657304, 0.8616798250174156] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 34.2171
Function value obtained: -0.3250
Current minimum: -0.4350
Iteration No: 3 started. Evaluating function at random point.
[0.004232013397179603, 68, 45, 0.2680983530433343, 0.5809725180523154] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 10.5448
Function value obtained: -0.2650
Current minimum: -0.4350
Iteration No: 4 started. Evaluating function at random point.
[0.0672858974212934, 60, 44, 0.9421713999524447, 0.8005503127028804] 

Iteration No: 4 ended. Evaluation done at random point.
Time

In [0]:
resultados_gp.x